In [ ]:
# !pip install langchain chromadb openai tiktoken pypdf langchain_openai langchain-community

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
import os
load_dotenv()


llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.2)

response = llm.invoke("what is machine learnig")

print(response.content)

In [5]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.vectorstores import Chroma

In [2]:
from langchain.schema import Document

# Create LangChain documents for IPL players

doc1 = Document(
        page_content="Virat Kohli is one of the most successful and consistent batsmen in IPL history. Known for his aggressive batting style and fitness, he has led the Royal Challengers Bangalore in multiple seasons.",
        metadata={"team": "Royal Challengers Bangalore"}
    )
doc2 = Document(
        page_content="Rohit Sharma is the most successful captain in IPL history, leading Mumbai Indians to five titles. He's known for his calm demeanor and ability to play big innings under pressure.",
        metadata={"team": "Mumbai Indians"}
    )
doc3 = Document(
        page_content="MS Dhoni, famously known as Captain Cool, has led Chennai Super Kings to multiple IPL titles. His finishing skills, wicketkeeping, and leadership are legendary.",
        metadata={"team": "Chennai Super Kings"}
    )
doc4 = Document(
        page_content="Jasprit Bumrah is considered one of the best fast bowlers in T20 cricket. Playing for Mumbai Indians, he is known for his yorkers and death-over expertise.",
        metadata={"team": "Mumbai Indians"}
    )
doc5 = Document(
        page_content="Ravindra Jadeja is a dynamic all-rounder who contributes with both bat and ball. Representing Chennai Super Kings, his quick fielding and match-winning performances make him a key player.",
        metadata={"team": "Chennai Super Kings"}
    )


In [3]:
docs = [doc1, doc2, doc3, doc4, doc5]

In [8]:
vector_store = Chroma(
    embedding_function=GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001"),
    persist_directory='my_chroma_db',
    collection_name='sample'
)

In [9]:
# add documents
vector_store.add_documents(docs)

['b4fae2de-67ea-424c-b99a-fafb693cab87',
 '8b80f5a0-8fc0-4381-87e7-312b03878de4',
 '9789c592-fbc1-4cee-8349-24f2f945e2df',
 '5bbe86ac-6fca-4c3e-9c47-c6277d2ae906',
 '3c3af042-786c-4b7a-9bd7-e51bf00e0b94']

In [10]:
# view documents
vector_store.get(include=['embeddings','documents', 'metadatas'])

{'ids': ['b4fae2de-67ea-424c-b99a-fafb693cab87',
  '8b80f5a0-8fc0-4381-87e7-312b03878de4',
  '9789c592-fbc1-4cee-8349-24f2f945e2df',
  '5bbe86ac-6fca-4c3e-9c47-c6277d2ae906',
  '3c3af042-786c-4b7a-9bd7-e51bf00e0b94'],
 'embeddings': array([[-0.00982054,  0.02545763,  0.02402782, ...,  0.01414876,
         -0.01560954, -0.00266117],
        [-0.01989721,  0.01092714,  0.01730603, ...,  0.00973945,
         -0.0176257 , -0.00460104],
        [-0.01358705, -0.00359449,  0.01163961, ...,  0.01149882,
         -0.02098301,  0.00373549],
        [-0.01261678, -0.00227585,  0.00470995, ..., -0.00144414,
          0.00646786, -0.00529741],
        [-0.01413488, -0.02750698,  0.01302837, ...,  0.00987475,
         -0.00950909, -0.00272136]], shape=(5, 3072)),
 'documents': ['Virat Kohli is one of the most successful and consistent batsmen in IPL history. Known for his aggressive batting style and fitness, he has led the Royal Challengers Bangalore in multiple seasons.',
  "Rohit Sharma is the m

In [11]:
# search documents
vector_store.similarity_search(
    query='Who among these are a bowler?',
    k=2
)

[Document(metadata={'team': 'Mumbai Indians'}, page_content='Jasprit Bumrah is considered one of the best fast bowlers in T20 cricket. Playing for Mumbai Indians, he is known for his yorkers and death-over expertise.'),
 Document(metadata={'team': 'Chennai Super Kings'}, page_content='Ravindra Jadeja is a dynamic all-rounder who contributes with both bat and ball. Representing Chennai Super Kings, his quick fielding and match-winning performances make him a key player.')]

In [12]:
# search with similarity score
vector_store.similarity_search_with_score(
    query='Who among these are a bowler?',
    k=2
)

[(Document(metadata={'team': 'Mumbai Indians'}, page_content='Jasprit Bumrah is considered one of the best fast bowlers in T20 cricket. Playing for Mumbai Indians, he is known for his yorkers and death-over expertise.'),
  0.6406819820404053),
 (Document(metadata={'team': 'Chennai Super Kings'}, page_content='Ravindra Jadeja is a dynamic all-rounder who contributes with both bat and ball. Representing Chennai Super Kings, his quick fielding and match-winning performances make him a key player.'),
  0.660536527633667)]

In [14]:
# meta-data filtering
vector_store.similarity_search_with_score(
    query="Who among these are a bowler?",
    filter={"team": "Chennai Super Kings"}
)

[(Document(metadata={'team': 'Chennai Super Kings'}, page_content='Ravindra Jadeja is a dynamic all-rounder who contributes with both bat and ball. Representing Chennai Super Kings, his quick fielding and match-winning performances make him a key player.'),
  0.660536527633667),
 (Document(metadata={'team': 'Chennai Super Kings'}, page_content='MS Dhoni, famously known as Captain Cool, has led Chennai Super Kings to multiple IPL titles. His finishing skills, wicketkeeping, and leadership are legendary.'),
  0.7242982387542725)]

In [15]:
# update documents
updated_doc1 = Document(
    page_content="Virat Kohli, the former captain of Royal Challengers Bangalore (RCB), is renowned for his aggressive leadership and consistent batting performances. He holds the record for the most runs in IPL history, including multiple centuries in a single season. Despite RCB not winning an IPL title under his captaincy, Kohli's passion and fitness set a benchmark for the league. His ability to chase targets and anchor innings has made him one of the most dependable players in T20 cricket.",
    metadata={"team": "Royal Challengers Bangalore"}
)

vector_store.update_document(document_id='09a39dc6-3ba6-4ea7-927e-fdda591da5e4', document=updated_doc1)

In [16]:
# view documents
vector_store.get(include=['embeddings','documents', 'metadatas'])

{'ids': ['b4fae2de-67ea-424c-b99a-fafb693cab87',
  '8b80f5a0-8fc0-4381-87e7-312b03878de4',
  '9789c592-fbc1-4cee-8349-24f2f945e2df',
  '5bbe86ac-6fca-4c3e-9c47-c6277d2ae906',
  '3c3af042-786c-4b7a-9bd7-e51bf00e0b94'],
 'embeddings': array([[-0.00982054,  0.02545763,  0.02402782, ...,  0.01414876,
         -0.01560954, -0.00266117],
        [-0.01989721,  0.01092714,  0.01730603, ...,  0.00973945,
         -0.0176257 , -0.00460104],
        [-0.01358705, -0.00359449,  0.01163961, ...,  0.01149882,
         -0.02098301,  0.00373549],
        [-0.01261678, -0.00227585,  0.00470995, ..., -0.00144414,
          0.00646786, -0.00529741],
        [-0.01413488, -0.02750698,  0.01302837, ...,  0.00987475,
         -0.00950909, -0.00272136]], shape=(5, 3072)),
 'documents': ['Virat Kohli is one of the most successful and consistent batsmen in IPL history. Known for his aggressive batting style and fitness, he has led the Royal Challengers Bangalore in multiple seasons.',
  "Rohit Sharma is the m

In [17]:
# delete document
vector_store.delete(ids=['09a39dc6-3ba6-4ea7-927e-fdda591da5e4'])

In [18]:
# view documents
vector_store.get(include=['embeddings','documents', 'metadatas'])

{'ids': ['b4fae2de-67ea-424c-b99a-fafb693cab87',
  '8b80f5a0-8fc0-4381-87e7-312b03878de4',
  '9789c592-fbc1-4cee-8349-24f2f945e2df',
  '5bbe86ac-6fca-4c3e-9c47-c6277d2ae906',
  '3c3af042-786c-4b7a-9bd7-e51bf00e0b94'],
 'embeddings': array([[-0.00982054,  0.02545763,  0.02402782, ...,  0.01414876,
         -0.01560954, -0.00266117],
        [-0.01989721,  0.01092714,  0.01730603, ...,  0.00973945,
         -0.0176257 , -0.00460104],
        [-0.01358705, -0.00359449,  0.01163961, ...,  0.01149882,
         -0.02098301,  0.00373549],
        [-0.01261678, -0.00227585,  0.00470995, ..., -0.00144414,
          0.00646786, -0.00529741],
        [-0.01413488, -0.02750698,  0.01302837, ...,  0.00987475,
         -0.00950909, -0.00272136]], shape=(5, 3072)),
 'documents': ['Virat Kohli is one of the most successful and consistent batsmen in IPL history. Known for his aggressive batting style and fitness, he has led the Royal Challengers Bangalore in multiple seasons.',
  "Rohit Sharma is the m